In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install timm

     |████████████████████████████████| 376 kB 5.4 MB/s 


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("ggplot")

import torch
import torch.nn as nn
import torchvision.transforms as transforms

import timm

import gc
import os
import time
import random
from datetime import datetime

from PIL import Image
from tqdm.notebook import tqdm
from sklearn import model_selection, metrics
from shutil import copyfile

In [4]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results
    
    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(1001)

In [5]:
!ls "gdrive/MyDrive/Machine Vision and Image Processing/Project/Dataset"

Pollen_data.zip  test.zip  train.zip


In [ ]:
!unzip -q "gdrive/MyDrive/Machine Vision and Image Processing/Project/Dataset/Pollen_data.zip"

## Loading the prepared Dataset

In [ ]:
# general global variables
DATA_PATH = "Pollen_data"
IMAGES_PATH = "Pollen_data/images"

In [ ]:
df = pd.read_csv(os.path.join(DATA_PATH, "data.csv"))
df.head()

In [ ]:
df.info()

In [ ]:
df.label.value_counts().plot(kind="bar")

In [ ]:
# Here for splitting the data into train, test and validation. We will using train_test_split from sklearn

# First divide the data into train data (80%) and remaining data(20%)
# Secodn divide the remaining data into validation (10%) and test data(10%)
train_df, remaining_df = model_selection.train_test_split(df, test_size=0.2, random_state=42, stratify=df.label.values)

In [ ]:
# Training data distribution
train_df.label.value_counts().plot(kind="bar")
plt.title("Training data distribution")

In [ ]:
train_df.label.value_counts()

In [ ]:
remaining_df.label.value_counts().plot(kind="bar")
plt.title("Except training data distribution")

In [ ]:
valid_df, test_df = model_selection.train_test_split(remaining_df, test_size=0.5, random_state=42, stratify=remaining_df.label.values)

In [ ]:
valid_df.label.value_counts()

In [ ]:
valid_df.label.value_counts().plot(kind="bar")
plt.title("Validation data distribution")

In [ ]:
!ls "gdrive/MyDrive/Machine Vision and Image Processing/Project/pretrained_model"

In [ ]:
# Getting the pretrained transformer model into the current running environment
!unzip -q "gdrive/MyDrive/Machine Vision and Image Processing/Project/pretrained_model/jx_vit_base_p16_224-80ecf9dd.pth.zip"

In [ ]:
MODEL_PATH = ("jx_vit_base_p16_224-80ecf9dd.pth")


# model specific global variables
IMG_SIZE = 224
BATCH_SIZE = 16
LR = 2e-05
GAMMA = 0.7
N_EPOCHS = 10

In [ ]:
DATA_PATH

In [34]:
len(df.values)

11279

In [103]:
class PollenDataset(torch.utils.data.Dataset):
    """
    Helper Class to create the pytorch dataset
    """

    def __init__(self, df, data_path=DATA_PATH, transforms=None):
        super().__init__()
        self.df_data = df.values
        self.data_path = data_path
        self.transforms = transforms
        self.data_dir = "images"

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        img_name, label = self.df_data[index]
        img_path = os.path.join(self.data_path, self.data_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            image = self.transforms(image)

        return image, torch.from_numpy(np.asarray(label, dtype=np.int64))

In [87]:
# Calculating the mean and standard deviation of my training df
# Then will use this information for normalization of the images in Compose
from torch.utils.data import DataLoader

class PollenDatasetDistributionCalculator(torch.utils.data.Dataset):
    """
    Helper Class to create the pytorch dataset
    """

    def __init__(self, df, data_path=DATA_PATH, transforms=None):
        super().__init__()
        self.df_data = df.values
        self.data_path = data_path
        self.transforms = transforms
        self.data_dir = "images"

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        img_name, label = self.df_data[index]
        img_path = os.path.join(self.data_path, self.data_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            image = self.transforms(image)

        return (torch.from_numpy(np.asarray(image, dtype=np.float32).transpose((2,0,1))), label)

train_dataset = PollenDatasetDistributionCalculator(train_df)
loader = DataLoader(train_dataset,
                          batch_size=10,
                          num_workers=1,
                          shuffle=False)

mean = 0.
std = 0.
nb_samples = 0.
for data, label in loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print(mean)
print(std)

tensor([161.8874, 134.1203,  92.8091])
tensor([54.6586, 54.8434, 44.0893])


In [104]:
# create image augmentations

# The Normalization metrics given that all the pixels are not rescaled i.e. are from 0 to 255
# The normalization

transforms_train = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.RandomResizedCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

transforms_valid = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

In [ ]:
print("Available Vision Transformer Models: ")
timm.list_models("vit*")

In [106]:
class ViTBase16(nn.Module):
    def __init__(self, n_classes, pretrained=False):

        super(ViTBase16, self).__init__()

        self.model = timm.create_model("vit_base_patch16_224", pretrained=False)
        if pretrained:
            self.model.load_state_dict(torch.load(MODEL_PATH))

        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        return x    

In [107]:
model = ViTBase16(n_classes=4, pretrained=True)

In [98]:
from torchsummary import summary
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
          Identity-2             [-1, 196, 768]               0
        PatchEmbed-3             [-1, 196, 768]               0
           Dropout-4             [-1, 197, 768]               0
         LayerNorm-5             [-1, 197, 768]           1,536
            Linear-6            [-1, 197, 2304]       1,771,776
           Dropout-7         [-1, 12, 197, 197]               0
            Linear-8             [-1, 197, 768]         590,592
           Dropout-9             [-1, 197, 768]               0
        Attention-10             [-1, 197, 768]               0
         Identity-11             [-1, 197, 768]               0
        LayerNorm-12             [-1, 197, 768]           1,536
           Linear-13            [-1, 197, 3072]       2,362,368
             GELU-14            [-1, 19

In [114]:
train_dataset = PollenDataset(train_df, transforms=transforms_train)
valid_dataset = PollenDataset(valid_df, transforms=transforms_valid)

In [115]:
train_loader = DataLoader(
     dataset=train_dataset,
     batch_size=BATCH_SIZE,
     shuffle = True
     )
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
    )

In [ ]:
epochs = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
check_every = 100
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
i = 1
for epoch in range(epochs):
  epoch_loss = 0.0
  epoch_accuracy = 0.0
  model.train()
  for data, target in train_loader:
    data, target = data.to(device, dtype=torch.float32), target.to(device, dtype=torch.int64)
    # clear the gradients of all optimized variables
    optimizer.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # Calculate Accuracy
    accuracy = (output.argmax(dim=1) == target).float().mean()
    # update training loss and accuracy
    epoch_loss += loss
    epoch_accuracy += accuracy
    
    optimizer.step()
    i += 1

  print("Training epoch loss : {} \t Training Accuracy : {}".format(epoch_loss / len(train_loader), epoch_accuracy / len(train_loader)))

  if i % check_every == 0:
      
    # keep track of validation loss
    valid_loss = 0.0
    valid_accuracy = 0.0

    ######################
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
      data, target = data.to(device, dtype=torch.float32), target.to(device, dtype=torch.int64)
      with torch.no_grad():
            # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # Calculate Accuracy
        accuracy = (output.argmax(dim=1) == target).float().mean()
        # update average validation loss and accuracy
        valid_loss += loss
        valid_accuracy += accuracy

    print("Validation loss : {} \t Validation Accuracy : {}".format(valid_loss / len(valid_loader), valid_accuracy / len(valid_loader)))

cpu


In [ ]:
def train_one_epoch(train_loader, criterion, optimizer, device):
        # keep track of training loss
        epoch_loss = 0.0
        epoch_accuracy = 0.0

        ###################
        # train the model #
        ###################
        self.model.train()
        for i, (data, target) in enumerate(train_loader):
            # move tensors to GPU if CUDA is available
            if device.type == "cuda":
                data, target = data.cuda(), target.cuda()
            elif device.type == "xla":
                data = data.to(device, dtype=torch.float32)
                target = target.to(device, dtype=torch.int64)

            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = self.forward(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # Calculate Accuracy
            accuracy = (output.argmax(dim=1) == target).float().mean()
            # update training loss and accuracy
            epoch_loss += loss
            epoch_accuracy += accuracy

            optimizer.step()

        return epoch_loss / len(train_loader), epoch_accuracy / len(train_loader)

    def validate_one_epoch(self, valid_loader, criterion, device):
        # keep track of validation loss
        valid_loss = 0.0
        valid_accuracy = 0.0

        ######################
        # validate the model #
        ######################
        self.model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            if device.type == "cuda":
                data, target = data.cuda(), target.cuda()
            elif device.type == "xla":
                data = data.to(device, dtype=torch.float32)
                target = target.to(device, dtype=torch.int64)

            with torch.no_grad():
                # forward pass: compute predicted outputs by passing inputs to the model
                output = self.model(data)
                # calculate the batch loss
                loss = criterion(output, target)
                # Calculate Accuracy
                accuracy = (output.argmax(dim=1) == target).float().mean()
                # update average validation loss and accuracy
                valid_loss += loss
                valid_accuracy += accuracy

        return valid_loss / len(valid_loader), valid_accuracy / len(valid_loader)

In [ ]:
def fit_tpu(
    model, epochs, device, criterion, optimizer, train_loader, valid_loader=None):

    valid_loss_min = np.Inf  # track change in validation loss

    # keeping track of losses as it happen
    train_losses = []
    valid_losses = []
    train_accs = []
    valid_accs = []

    for epoch in range(1, epochs + 1):
        gc.collect()
        # para_train_loader = pl.ParallelLoader(train_loader, [device])

        # xm.master_print(f"{'='*50}")
        # xm.master_print(f"EPOCH {epoch} - TRAINING...")
        train_loss, train_acc = model.train_one_epoch(
            para_train_loader.per_device_loader(device), criterion, optimizer, device
        )
        xm.master_print(
            f"\n\t[TRAIN] EPOCH {epoch} - LOSS: {train_loss}, ACCURACY: {train_acc}\n"
        )
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        gc.collect()

        if valid_loader is not None:
            gc.collect()
            para_valid_loader = pl.ParallelLoader(valid_loader, [device])
            xm.master_print(f"EPOCH {epoch} - VALIDATING...")
            valid_loss, valid_acc = model.validate_one_epoch(
                para_valid_loader.per_device_loader(device), criterion, device
            )
            xm.master_print(f"\t[VALID] LOSS: {valid_loss}, ACCURACY: {valid_acc}\n")
            valid_losses.append(valid_loss)
            valid_accs.append(valid_acc)
            gc.collect()

            # save model if validation loss has decreased
            if valid_loss <= valid_loss_min and epoch != 1:
                xm.master_print(
                    "Validation loss decreased ({:.4f} --> {:.4f}).  Saving model ...".format(
                        valid_loss_min, valid_loss
                    )
                )
            #                 xm.save(model.state_dict(), 'best_model.pth')

            valid_loss_min = valid_loss

    return {
        "train_loss": train_losses,
        "valid_losses": valid_losses,
        "train_acc": train_accs,
        "valid_acc": valid_accs,
    }

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def _run():
    train_dataset = CassavaDataset(train_df, transforms=transforms_train)
    valid_dataset = CassavaDataset(valid_df, transforms=transforms_valid)

    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True,
    )

    valid_sampler = torch.utils.data.distributed.DistributedSampler(
        valid_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=False,
    )

    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        sampler=train_sampler,
        drop_last=True,
        num_workers=8,
    )

    valid_loader = torch.utils.data.DataLoader(
        dataset=valid_dataset,
        batch_size=BATCH_SIZE,
        sampler=valid_sampler,
        drop_last=True,
        num_workers=8,
    )

    criterion = nn.CrossEntropyLoss()
    #     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = xm.xla_device()
    model.to(device)

    lr = LR * xm.xrt_world_size()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    xm.master_print(f"INITIALIZING TRAINING ON {xm.xrt_world_size()} TPU CORES")
    start_time = datetime.now()
    xm.master_print(f"Start Time: {start_time}")

    logs = fit_tpu(
        model=model,
        epochs=N_EPOCHS,
        device=device,
        criterion=criterion,
        optimizer=optimizer,
        train_loader=train_loader,
        valid_loader=valid_loader,
    )

    xm.master_print(f"Execution time: {datetime.now() - start_time}")

    xm.master_print("Saving Model")
    xm.save(
        model.state_dict(), f'model_5e_{datetime.now().strftime("%Y%m%d-%H%M")}.pth'
    )

<IPython.core.display.Javascript object>

In [ ]:
# Start training processes
def _mp_fn(rank, flags):
    torch.set_default_tensor_type("torch.FloatTensor")
    a = _run()


# _run()
FLAGS = {}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method="fork")

INITIALIZING TRAINING ON 8 TPU CORES
Start Time: 2020-12-02 18:34:26.339168
EPOCH 1 - TRAINING...
	BATCH 1/150 - LOSS: 1.609375
	BATCH 21/150 - LOSS: 0.7734375
	BATCH 41/150 - LOSS: 0.33984375
	BATCH 61/150 - LOSS: 0.333984375
	BATCH 81/150 - LOSS: 0.314453125
	BATCH 101/150 - LOSS: 0.765625
	BATCH 121/150 - LOSS: 0.439453125
	BATCH 141/150 - LOSS: 0.451171875

	[TRAIN] EPOCH 1 - LOSS: 0.59765625, ACCURACY: 0.83203125

EPOCH 1 - VALIDATING...
	[VALID] LOSS: 0.419921875, ACCURACY: 0.84375

EPOCH 2 - TRAINING...
	BATCH 1/150 - LOSS: 0.458984375
	BATCH 21/150 - LOSS: 0.2734375
	BATCH 41/150 - LOSS: 0.1396484375
	BATCH 61/150 - LOSS: 0.177734375
	BATCH 81/150 - LOSS: 0.14453125
	BATCH 101/150 - LOSS: 0.5703125
	BATCH 121/150 - LOSS: 0.32421875
	BATCH 141/150 - LOSS: 0.60546875

	[TRAIN] EPOCH 2 - LOSS: 0.41796875, ACCURACY: 0.9140625

EPOCH 2 - VALIDATING...
	[VALID] LOSS: 0.4140625, ACCURACY: 0.83984375

Validation loss decreased (0.4199 --> 0.4141).  Saving model ...
EPOCH 3 - TRAINING..

<IPython.core.display.Javascript object>

## Thanks a lot for reading all the way

# <font size=4 color='blue'>If you find this notebook useful, leave an upvote, that motivates me to write more such notebooks.</font>